##### Step 1 - Read the CSV file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType


In [0]:
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True) ])

In [0]:
storage_account_name = "formula1dl099"
container_name = "raw"
csv_file_name = "races.csv"

csv_file_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{csv_file_name}"


In [0]:
races_df = spark.read.csv(csv_file_path, header=True)

##### Step 2 - Add ingestion date and race_timestamp to the dataframe

In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp, concat, col, lit


In [0]:
races_with_timestamp_df = races_df.withColumn("ingestion_date", current_timestamp()) \
                                  .withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss'))

##### Step 3 - Select only the columns required & rename as required





In [0]:
races_selected_df = races_with_timestamp_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                   col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('race_timestamp'))

In [0]:
races_selected_df.show()

+-------+---------+-----+----------+--------------------+--------------------+-------------------+
|race_id|race_year|round|circuit_id|                name|      ingestion_date|     race_timestamp|
+-------+---------+-----+----------+--------------------+--------------------+-------------------+
|      1|     2009|    1|         1|Australian Grand ...|2023-12-24 18:35:...|2009-03-29 06:00:00|
|      2|     2009|    2|         2|Malaysian Grand Prix|2023-12-24 18:35:...|2009-04-05 09:00:00|
|      3|     2009|    3|        17|  Chinese Grand Prix|2023-12-24 18:35:...|2009-04-19 07:00:00|
|      4|     2009|    4|         3|  Bahrain Grand Prix|2023-12-24 18:35:...|2009-04-26 12:00:00|
|      5|     2009|    5|         4|  Spanish Grand Prix|2023-12-24 18:35:...|2009-05-10 12:00:00|
|      6|     2009|    6|         6|   Monaco Grand Prix|2023-12-24 18:35:...|2009-05-24 12:00:00|
|      7|     2009|    7|         5|  Turkish Grand Prix|2023-12-24 18:35:...|2009-06-07 12:00:00|
|      8| 

##### Step 4 - Write the output to processed container in parquet format





In [0]:
races_selected_df.write.mode('overwrite').partitionBy('race_year').parquet('abfss://processed@formula1dl099.dfs.core.windows.net/races')